# Human in the loop  (HITL) with Strands Agents on AgentCore Runtime with Anthropic Claude Sonnet 4.5

## Overview

This tutorial will go over how to host an agent with tools that require human approval, using Amazon Bedrock AgentCore Runtime & Anthropic Claude Sonnet 4.5.

### Tutorial Details


| Information         | Details                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| Tutorial type       | Conversational                                                                   |
| Agent type          | Single                                                                           |
| Agentic Framework   | Strands Agents                                                                   |
| LLM model           | Anthropic Claude Sonnet 4.5                                                      |
| Tutorial components | Hosting agent on AgentCore Runtime, using a Strands Agent with HITL              |
| Tutorial vertical   | Cross-vertical                                                                   |
| Example complexity  | Medium                                                                           |
| SDK used            | Amazon BedrockAgentCore Python SDK and boto3                                     |

### Tutorial Architecture

In this tutorial we will describe how to create an Agent with tools protected by human approval. This will be deployed to AgentCore runtime. 

For demonstration purposes, we will  use a Strands Agent using Anthropic Claude Sonnet 4.5.

In our example we will use a very simple agent with two tools: `send_email` and `get_weather`. 

Strand's inherent tool `handoff_to_user` will be used to intercept tool calls in the agent loop. 

<div style="text-align:left">
    <img src="images/architecture_runtime.jpg" width="50%"/>
</div>

### Tutorial Key Features

* Hosting Strands Agents on Amazon Bedrock AgentCore Runtime
* Using Anthropic Claude Sonnet 4.5
* Using Strands Agents built in "handoff_to_user" functionality for human-in-the-loop


Install dependencies

In [ ]:
!pip install -U --quiet boto3 bedrock-agentcore-starter-toolkit bedrock-agentcore strands-agents strands-agents-tools

Create requirements file for AgentCore Runtime deployment

In [ ]:
%%writefile requirements.txt
boto3
bedrock-agentcore-starter-toolkit
bedrock-agentcore
strands-agents
strands-agents-tools

## Preparing our HITL Agent for Deployment on AgentCore Runtime

Let's now deploy our **HITL** Strands Agent to AgentCore Runtime.  
This agent demonstrates how to **pause tool execution** and **hand control back to a human** directly from within the agent loop.

---

### Key points for HITL implementation:

- Use an **approval-aware proxy tool** (e.g., `send_email`) that checks if the action is approved before running.  
- Maintain a **protected tools list** (`PROTECTED_TOOLS`) to define which actions need explicit approval.  
- Read **approval flags** from the runtime payload (`payload["approvals"]`).  
- If approval is missing or denied, the tool **returns a `handoff_to_user`** response to pause execution.  
- Unprotected tools (like `get_weather`) execute automatically without intervention.  

---

## Inside the Strands Agent: Approval-Aware Tools

This HITL pattern is implemented **directly inside the tool itself**, without relying on external hooks or helper functions.

The `send_email` tool performs both the **approval check** and the **execution logic**:
- When approval exists (`approvals["send_email"] == True`), it immediately performs the email action and returns a success message.  
- When approval is missing or denied, it **returns a `handoff_to_user` response** that pauses execution and asks for human approval.  
- The agent then surfaces this message verbatim to the user and waits for confirmation before retrying.

---

## Understanding Human-in-the-Loop in AgentCore Runtime

When using HITL patterns with Strands + AgentCore, the following behaviors occur automatically:

### Approval Flow
- The runtime payload can include an `approvals` object specifying tool permissions.  
- Each proxy tool checks these flags before proceeding with execution.  

### Handoff to User
- If approval is not granted, the tool calls `handoff_to_user(message=..., reason=...)`.  
- This **pauses execution inside the agent loop**, signaling the need for human review.  
- The user can then re-invoke the same prompt with `approvals["tool_name"] = true` to proceed. 

In [ ]:
%%writefile hitl_strands_agentcore.py
#Imports
from typing import Any, Dict, Iterable, Callable, Mapping, Optional
from strands import Agent, tool
from strands.models import BedrockModel
from strands.hooks import HookProvider, HookRegistry
from strands.experimental.hooks import BeforeToolInvocationEvent
from strands_tools import handoff_to_user
from bedrock_agentcore.runtime import BedrockAgentCoreApp

# AgentCore Runtime app
app = BedrockAgentCoreApp()

# Choose your Bedrock model
MODEL_ID = "us.anthropic.claude-sonnet-4-5-20250929-v1:0"

# Protected tool list. Any tool listed here requires explicit approval
PROTECTED_TOOLS = {"send_email"}

# Keep the latest payload accessible to tools/hooks
last_payload: Dict[str, Any] = {}

# Track payload for Agent loop (so it can decide to handoff to user or not)
def payload_get() -> Dict[str, Any]:
    """Return the most recent invocation payload."""
    return last_payload or {}

@tool(description="Send a transactional email. Requires human approval before execution.")
def send_email(to: str, subject: str, body: str) -> dict:
    """
    This tool checks whether sending an email is approved.
    If approved, it executes the action.
    Otherwise, it returns a handoff_to_user event inside the agent loop.
    """
    approvals = (payload_get().get("approvals") or {})

    # Approved -> perform the real action
    if approvals.get("send_email") is True:
        return {"content": [{"text": f"Email successfully sent to {to!r} with subject {subject!r}"}]}

    # Not approved -> hand back to user
    reason = "approval_required" if "send_email" not in approvals else "approval_denied"
    msg = (
        "Execution of 'send_email' requires human approval. "
        "Re-invoke with approvals['send_email']=true to proceed."
        if reason == "approval_required"
        else "The request to execute 'send_email' was denied by the user. No action was taken."
    )
    return handoff_to_user(message=msg, reason=reason)

@tool(description="Return weather.")
def get_weather() -> str:
    # Demo only
    return "sunny"

# Create Strands Agent
model = BedrockModel(model_id=MODEL_ID)
agent = Agent(
    model=model,
    tools=[send_email, get_weather, handoff_to_user], # handoff_to_user (built into Strands) must be added to the tools list 
    system_prompt=( #Prompt engineering is an art & a science, ensure this is optimized for your usecase. This one is optimized for this notebook's purposes
        "You are a helpful assistant.\n"
        "Use tools to perform actions (sending email & checking weather).\n"
        "When a user asks to send an email, you MUST call the send_email tool.\n"
        "Do NOT claim an action occurred unless a tool_result confirms it.\n"
        "If 'handoff_to_user' is used, OUTPUT THE HANDOFF MESSAGE VERBATIM and STOP. "
        "Do not apologize or call it a technical error.\n"
        "For protected actions without explicit approval, call 'handoff_to_user'."
    )
)

# Entrypoint
@app.entrypoint
def strands_agent_hitl(payload):
    """
    Payload example:
      {
        "prompt": "send an email to dev@example.com about launch features",
        "approvals": {"send_email": true}  # optional per-tool approval flags
      }
    """
    global last_payload
    last_payload = payload or {}

    user_input = (payload or {}).get("prompt", "")
    if not user_input:
        return "Please provide a 'prompt'."

    response = agent(user_input)

    # Extract text
    try:
        return response.message["content"][0]["text"]
    except Exception:
        return str(getattr(response, "text", response))


if __name__ == "__main__":
    app.run()


### Configure AgentCore Runtime deployment

First we will use our starter toolkit to configure the AgentCore Runtime deployment with an entrypoint, the execution role we just created and a requirements file. We will also configure the starter kit to auto create the Amazon ECR repository on launch.

During the configure step, your docker file will be generated based on your application code

<div style="text-align:left">
    <img src="images/configure.png" width="60%"/>
</div>

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session

boto_sess = Session()
region = boto_sess.region_name

agentcore_runtime = Runtime()
agent_name = "hitl_strands_bedrock_demo"      # <-- Change as needed

response = agentcore_runtime.configure(
    entrypoint="hitl_strands_agentcore.py",   # <-- The file created above
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",     # <-- The requirements file created above
    region=region,
    agent_name=agent_name
)
response

### Launching agent to AgentCore Runtime

Now that we've got a docker file, let's launch the agent to the AgentCore Runtime. This will create the Amazon ECR repository and the AgentCore Runtime

<div style="text-align:left">
    <img src="images/launch.png" width="75%"/>
</div>

In [ ]:
#Launch the Runtime
launch_result = agentcore_runtime.launch()
launch_result

### Checking for the AgentCore Runtime Status
Now that we've deployed the AgentCore Runtime, let's check for it's deployment status

In [ ]:
#Wait for the Runtime status as READY
import time

status_response = agentcore_runtime.status()
status = status_response.endpoint["status"]
end_status = ["READY", "CREATE_FAILED", "DELETE_FAILED", "UPDATE_FAILED"]

while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint["status"]
    print(status)

status

### Invoking AgentCore Runtime

Finally, we can invoke our AgentCore Runtime with a payload

<div style="text-align:left">
    <img src="images/invoke.png" width=75%"/>
</div>

In [ ]:
#Function to parse response text from AgentCore
#If you want to see the output metadata as well, just print "invoke_response" in the invocation cells
from IPython.display import Markdown, display
import json
def parse_response(invoke_response):
    response_text = invoke_response['response'][0]
    return display(Markdown(response_text))

In [ ]:
#Invoke with no approval a False flag is optional, the Agent will handoff to user as long as the payload does not have "True"
invoke_response = agentcore_runtime.invoke({
    "prompt": "Can you send an email for me. I want to send it to dev@example.com asking about what features will be available at launch, draft it end to end before sending",
    #"approvals": {"send_email": False}
})
parse_response(invoke_response)

In [ ]:
#Invoke with approval 
invoke_response = agentcore_runtime.invoke({
    "prompt": "The email is approved",
    "approvals": {"send_email": True}
})
parse_response(invoke_response)

In [ ]:
#Invoke unprotected tool no approvals needed
invoke_response = agentcore_runtime.invoke({
    "prompt": "What is the weather"
})
parse_response(invoke_response)

## Cleanup (Optional)

Let's now clean up the AgentCore Runtime created

In [ ]:
#Cleanup (Optional) -> Uncomment the rest of this cell to cleanup the resources created
# import boto3

# agent_id = launch_result.agent_id
# ecr_uri = launch_result.ecr_uri
# repo_name = ecr_uri.split("/")[1]

# control = boto3.client("bedrock-agentcore-control", region_name=region)
# ecr = boto3.client("ecr", region_name=region)

# tmp = control.delete_agent_runtime(agentRuntimeId=agent_id)
# tmp = ecr.delete_repository(repositoryName=repo_name, force=True)

## Conclusion

In this notebook, we built an end-to-end **Human-in-the-Loop** agent using **Strands + Anthropic Claude Sonnet 4.5 + AgentCore Runtime**, demonstrating how AI-driven actions can remain transparent, safe, and human-approved.

### Some Takeaways

- **Embedded control logic:** By placing approval checks directly inside the tool, the agent ensures human consent is required for sensitive operations without needing external hooks.  
- **Seamless user experience:** The use of `handoff_to_user` from Strands allows the model to pause gracefully.
- **Flexible runtime design:** AgentCore Runtime automatically manages session context and payloads, making it easy to pass approval metadata between invocations.  
- **Scalable pattern:** This same structure can be extended to other sensitive tools (e.g., `delete_user`, `approve_invoice`, etc.) by following the same approval-aware proxy pattern. 

This foundation can be expanded into larger multi-agent systems, approval workflows, or enterprise-grade GenAI governance frameworks — ensuring **responsible autonomy** while maintaining **human oversight** at every step.
